In [100]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import os
import json
import datetime
import pandas as pd
import enchant
import numpy as np
from nltk.stem.snowball import SnowballStemmer
stemmer = LancasterStemmer()
df_train = pd.read_csv("/home/yassaman/git/CMPE297/FinalProject/test_mini.csv").fillna("empty")
df_train.question1 = df_train.question1.astype(str)
df_train.question2 = df_train.question2.astype(str)

In [101]:
import string
import re
from string import digits
from nltk.corpus import stopwords
import enchant


def spell_check(w):
    d = enchant.Dict("en_US")
    if(d.check(w)==False):
        if(d.suggest(w) is not None):
            try:
                w = d.suggest(w)[0]
            except IndexError:
                w=w
                print 'Error ' + w
    return w

words = []
document1 = []
document2 = []
stop = set(stopwords.words('english'))
for index, row in df_train.iterrows():
    row['question1'].translate(None, string.punctuation)
    row['question2'].translate(None, string.punctuation)
    row['question1'] = re.sub(r"[^A-Za-z0-9]", " ", row['question1'])
    row['question2'] = re.sub(r"[^A-Za-z0-9]", " ", row['question2'])
    row['question1'] = re.sub('[!\[\]@#\\\.?,$/]', '', row['question1'])
    row['question2'] = re.sub('[!\[\]@#\\\.?,$/]', '', row['question2'])
#     remove_digits = str.maketrans('', '', digits)
    row['question1'] = row['question1'].translate(None, digits)
    row['question2'] = row['question2'].translate(None, digits)
    word_list = nltk.word_tokenize(row['question1'])
    word_list2 = nltk.word_tokenize(row['question2'])
    # add to our words list
    sentence1= []
    sentence2= []
    for w in word_list:
        w = SnowballStemmer("english").stem(w)
        w = spell_check(w)
        if w not in stop:
            sentence1.append(w.lower())
    # add to documents in our corpus
    str1 = ' '.join(sentence1)
    document1.append(str1)
    for w2 in word_list2:
        w2 = SnowballStemmer('english').stem(w2)
        w2 = spell_check(w2)
        if w2 not in stop:
            sentence2.append(w2.lower())
    # add to documents in our corpus
    str2 = ' '.join(sentence2)
    document2.append(str2)
# words = list(set(words))

In [109]:
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.snowball import SnowballStemmer
np.seterr(divide='ignore', invalid='ignore')
stemmer = LancasterStemmer()
def words_in_string(word_list, a_string):
    return set(word_list).intersection(a_string.split())
def cosine(a0,a1):
    dotp = np.dot(a0, a1)
    mag0 = np.linalg.norm(a0)
    mag1 = np.linalg.norm(a1)
    cosineSim = dotp / (mag0* mag1)
    return cosineSim
cosinesimilarityarray=[]
# print document1
# print '*****'
# print document2
merged_documents = document1 + document2
# print merged_documents
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(merged_documents)
# tfidf_matrix2 = tfidf_vectorizer.fit_transform(document2)
# print tfidf_matrix.shape
# print tfidf_matrix.shape[0]

num_questions = tfidf_matrix.shape[0]/2
for i in range(num_questions):
    index = i+1
    sim_array = cosine_similarity(tfidf_matrix[0:index], tfidf_matrix[0:index+num_questions])
    print sim_array
    np.savetxt("similarity_matrix.csv", sim_array, fmt="%.3g", delimiter=",")
# for a,b in zip(document1,document2):
#     print a + ' , ' + b
#     corpus = []
#     newCorpus = []
#     for i,j in zip(a,b):
# #         print '----'
# #         print a
#         i = SnowballStemmer("english").stem(i)
#         j = SnowballStemmer("english").stem(j)
# #     print a[0]
#     corpus.append(a[0])
#     corpus.append(b[0])
#     corpus[0] = ' '.join(corpus[0]).lower()
#     corpus[1] = ' '.join(corpus[1]).lower()
#     punc_trans_table = {ord(c): None for c in string.punctuation}
#     corpus[0].translate(punc_trans_table)
#     corpus[1].translate(punc_trans_table)
#     corpus[0]= re.sub('[!\[\]@#\\\.:?)(}{,$/]', '', corpus[0])
#     corpus[1]= re.sub('[!\[\]@#\\\.:?)(}{,$/]', '', corpus[1])
# #     remove_digits = string.maketrans('', '', digits)
#     digit_trans_table = {ord(c): None for c in string.digits}
#     corpus[0] = corpus[0].translate(digit_trans_table)
#     corpus[1] = corpus[1].translate(digit_trans_table)
# #     string0 = words_in_string(words,corpus[0])
# #     string1 = words_in_string(words,corpus[1])
#     newCorpus.append(corpus[0])
#     newCorpus.append(corpus[1])
#     vectorizer = CountVectorizer(min_df=1) 
#     analyze = vectorizer.build_analyzer()
#     print newCorpus
#     X = vectorizer.fit_transform(newCorpus)
#     xArray = X.toarray()
#     cosinesimilarity = cosine(xArray[0],xArray[1])
#     cosinesimilarityarray.append(cosinesimilarity)

[[ 1.          0.          0.          0.          0.          0.          0.
   0.20497797  0.          0.          0.          0.          0.          0.
   0.          0.          0.92853584]]
[[ 1.          0.          0.          0.          0.          0.          0.
   0.20497797  0.          0.          0.          0.          0.          0.
   0.          0.          0.92853584  0.        ]
 [ 0.          1.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.52378585]]
[[ 1.          0.          0.          0.          0.          0.          0.
   0.20497797  0.          0.          0.          0.          0.          0.
   0.          0.          0.92853584  0.          0.        ]
 [ 0.          1.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.         

In [103]:
len(cosinesimilarityarray)

0

In [104]:
import enchant
d = enchant.Dict("en_US")
for w in words:
    if(d.check(w)==False):
        if(d.suggest(w) is not None):
            try:
                w = d.suggest(w)[0]
            except IndexError:
                w=w